In [0]:
# tiers = ["bronze","silver","gold"]
# adl_paths = {tier : f'abfss://{tier}@myfirststorageaccount271.dfs.core.windows.net/' for tier in tiers}

# bronze_adls = adl_paths['bronze']
# silver_adls = adl_paths['silver']
# gold_adls = adl_paths['gold']

# dbutils.fs.ls(bronze_adls)
# dbutils.fs.ls(silver_adls)
# dbutils.fs.ls(gold_adls)
# silver_data_path = f"{silver_adls}earthquake_events_silver/"


# import requests
# import json
# from datetime import datetime
# from datetime import date, timedelta

# start_date = date.today() - timedelta(1)
# end_date = date.today()

#### Fetch data from Silver and Bronze Tasks

In [0]:
bronze_output = dbutils.jobs.taskValues.get(taskKey='Bronze',key='bronze_output')
silver_output = dbutils.jobs.taskValues.get(taskKey='Silver',key='silver_output')

start_date = bronze_output['start_date']
end_date = bronze_output['end_date']

bronze_adls = bronze_output['bronze_adls']
silver_adls = bronze_output['silver_adls']
gold_adls = bronze_output['gold_adls']

print(f'Start Date {start_date}, End Date {end_date}')

In [0]:
from pyspark.sql.functions import when,col, udf
from pyspark.sql.types import StringType

import reverse_geocoder as rg
from datetime import date, timedelta

In [0]:
df = spark.read.parquet(silver_output).filter(col('time') > start_date)


#### Add country code given {long,lat}

In [0]:
df = df.limit(1000) # shorten for testing

def get_country_code(lat,lon):
    try : 
        coordinates = (float(lat),float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(result)
        print(f"Processsed coordinates: ,{coordinates}, result: {result}")
        return result
    except Exception as e :
        print(f'Error converting coordinates to country code : {e}')
              

In [0]:
country_code_udf = udf(get_country_code,StringType())
df_with_location = df.withColumn("country_code",country_code_udf(col('latitude'),col('longitude')))

#### Add Singificance Class 

In [0]:
df_with_location = \
    df_with_location.\
        withColumn('sig_class',
                   when(col('sig') < 100,'Low').\
                   when(col('sig') >= 100, 'Medium').\
                   otherwise('High'))


#### Save as Parquet File

In [0]:
### Gold Output Path
gold_output_path = f"{gold_adls}earthquake_events_gold/"
df_with_location.write.mode('append').parquet(gold_output_path)